### Progetto 8b
Luca Maria Lauricella 519997

Valerio Marini 512489

Repository del progetto:
https://github.com/lauriluca99/TGW-3D.jl

Documentazione del progetto:
https://lauriluca99.github.io/TGW-3D.jl

In [1]:
#Importing all the libraries used in the notebooks
using Profile
using ProfileView
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

    function frag_face(
            V::Points, 
            EV::ChainOp, 
            FE::ChainOp, 
	        sp_idx::Vector{Int64}, 
            sigma::Int64)
            
Prende la faccia `sigma` e la trasforma in 2D per poter calcolare le intersezioni con le facce in `sp_idx[sigma]`
ed ottenere la disposizione 2D della faccia `sigma`.


### Versione iniziale di frag_face

In [2]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)
    

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
       

        
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

In [3]:
V = Float64[
    0 0 0; 0 1 0;
    1 1 0; 1 0 0;
    0 0 1; 0 1 1;
    1 1 1; 1 0 1
]

8×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  1.0  0.0
 1.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  1.0
 1.0  1.0  1.0
 1.0  0.0  1.0

In [4]:
EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
])

12×8 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
 -1   1   ⋅   ⋅   ⋅   ⋅   ⋅  ⋅
  ⋅  -1   1   ⋅   ⋅   ⋅   ⋅  ⋅
  ⋅   ⋅  -1   1   ⋅   ⋅   ⋅  ⋅
 -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  ⋅
 -1   ⋅   ⋅   ⋅   1   ⋅   ⋅  ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1   ⋅  ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  ⋅
  ⋅   ⋅   ⋅  -1   ⋅   ⋅   ⋅  1
  ⋅   ⋅   ⋅   ⋅  -1   1   ⋅  ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅  -1   1  ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  1
  ⋅   ⋅   ⋅   ⋅  -1   ⋅   ⋅  1

In [5]:
FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

`@btime` riporta i risultati come `@time`, mostrando le allocazioni ed il tempo minimo trascorso in una singola chiamata di funzione durante l'esecuzione della funzione su più campionamenti. Useremo `@btime` nella maggior parte dei nostri esempi.

In [6]:
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

  352.200 μs (4870 allocations: 265.34 KiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 1, 2, 2, 3, 3, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, 1, -1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, 1, 1, -1], 1, 4))

Utilizzando la macro `@code_warntype` si individuano molte variabili assegnate al tipo `Any`. Questo significa essenzialmente che ci sarà un'allocazione per la posizione della memoria e l'indirezione al valore effettivo durante l'esecuzione della funzione. 

In [7]:
@code_warntype frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

MethodInstance for frag_face(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Int64}, ::Int64)
  from frag_face(V, EV, FE, sp_idx, sigma) in Main at c:\Users\Valerio\Desktop\TGW-3D.jl\notebooks\frag_face.ipynb:1
Arguments
  #self#::Core.Const(frag_face)
  V::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
  sp_idx::Vector{Int64}
  sigma::Int64
Locals
  @_7::Any
  @_8::Union{Nothing, Tuple{Int64, Nothing}}
  nvsize::Any
  nFE::Any
  nEV::Any
  nV::Any
  tV::Any
  M::Any
  sEV::Any
  sV::Any
  sigmavs::Vector{Int64}
  vs_num::Int64
  @_19::Any
  @_20::Any
  i::Int64
  tmpEV::Any
  tmpV::Any
Body::Tuple{Any, Any, Any}
1 ─       Core.NewvarNode(:(@_7))
│         Core.NewvarNode(:(nvsize))
│         Core.NewvarNode(:(nFE))
│         Core.NewvarNode(:(nEV))
│         Core.NewvarNode(:(nV))
│         (vs_num = Main.size(V, 1))
│   %7  = (sigma:sigma)::UnitRange{Int64}
│   %8  = Base.getindex(FE, %7, Main.:(:))::S

Utilizzando la macro `@profile` si immagazzinano le informazioni relative alla funzione che successivamente vengono mostrate graficamente. Tramite il grafico riusciamo a dare una misurazione temporale di ogni singola funzione. La larghezza delle barre mostra il tempo trascorso in ogni locazione di chiamata, mentre la gerarchia di chiamata è rappresentata dalle varie altezze del grafico.

In [8]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    @profile frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [9]:
ProfileView.view()
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
 ╎47 @Base\client.jl:495; _start()
 ╎ 47 @Base\client.jl:292; exec_options(opts::Base.JLOptions)
 ╎  47 @Base\Base.jl:418; include(mod::Module, _path::String)
 ╎   47 @Base\loading.jl:1253; _include(mapexpr::Function, mod...
 ╎    47 @Base\loading.jl:1196; include_string(mapexpr::typeof(...
 ╎     47 @Base\boot.jl:373; eval
 ╎    ╎ 47 ...otebook\notebook.jl:32; top-level scope
 ╎    ╎  47 ...c\serve_notebook.jl:75; (::VSCodeServer.var"#serve_not...
 ╎    ╎   47 ...\serve_notebook.jl:136; serve_notebook(pipename::Stri...
 ╎    ╎    47 @JSONRPC\src\typed.jl:67; dispatch_msg(x::VSCodeServer....
 ╎    ╎     47 ...\serve_notebook.jl:13; notebook_runcell_request(con...
 ╎    ╎    ╎ 47 ...erver\src\repl.jl:184; withpath(f::VSCodeServer.va...
 ╎    ╎    ╎  47 ...serve_notebook.jl:19; (::VSCodeServer.var"#164#16...
 ╎    ╎    ╎   47 @Base\essentials.jl:714; invokelatest
 ╎    ╎    ╎    47 @Base\essentials.jl:716; #invokelatest#2
 ╎    ╎ 


 ╎    ╎    ╎    ╎    ╎    2  ...rangement.jl:200; biconnected_compone...
 ╎    ╎    ╎    ╎    ╎     2  ...angement.jl:173; an_edge
 ╎    ╎    ╎    ╎    ╎    ╎ 1  @Base\array.jl:2637; setdiff
 ╎    ╎    ╎    ╎    ╎    ╎  1  ...e\array.jl:2633; _shrink(shrinker!...
 ╎    ╎    ╎    ╎    ╎    ╎   1  ...e\array.jl:2602; _shrink_filter!
1╎    ╎    ╎    ╎    ╎    ╎    1  ...\array.jl:2592; _unique_filter!
 ╎    ╎    ╎    ╎    ╎    ╎ 1  ...sevector.jl:557; getindex(x::Sparse...
 ╎    ╎    ╎    ╎    ╎    ╎  1  @Base\array.jl:869; getindex
 ╎    ╎    ╎    ╎    ╎    ╎   1  ...ctarray.jl:783; similar
 ╎    ╎    ╎    ╎    ╎    ╎    1  ...e\array.jl:378; similar
 ╎    ╎    ╎    ╎    ╎    ╎     1  ...e\boot.jl:466; Array
1╎    ╎    ╎    ╎    ╎    ╎    ╎ 1  ...e\boot.jl:457; Array
 ╎    ╎    ╎    ╎    ╎   17 ...rangement.jl:551; planar_arrangement_2...
 ╎    ╎    ╎    ╎    ╎    1  ...rangement.jl:408; componentgraph(V::M...
 ╎    ╎    ╎    ╎    ╎     1  ...sematrix.jl:2169; getindex
 ╎    ╎    ╎    ╎

# Ottimizzazione



Possiamo assegnare alle variabili locali un tipo deterministico per rimuovere il tipo `Any`.
Inoltre si possono creare delle viste degli array, con la macro `@views`, che permettono di accedere ai valori dell'array senza dover effettuare una copia.

Si nota come utilizzando le macro `@views` e `@async` la funzione frag_face ottiene un nuovo tempo minimo con un decremento di circa 20% rispetto alla funzione originale.

In [10]:
function frag_face2(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, 
    sp_idx::Vector{Int64}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
    @views tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    @async for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    @views sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    @views nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]
    
    return nV, nEV, nFE
end

frag_face2 (generic function with 1 method)

#### Tempi minimi ottenuti da `@btime` con le diverse macro:

`@async` = 281.500 μs (3873 allocations: 218.65 KiB)

`Threads.@spawn` = 305.200 μs (4093 allocations: 230.38 KiB)

`@simd` = 383.000 μs (4996 allocations: 269.99 KiB)

`@inbounds` = 377.700 μs (4996 allocations: 269.99 KiB)

In [11]:
@btime frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

  277.700 μs (3873 allocations: 218.65 KiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 1, 2, 2, 3, 3, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, 1, -1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, 1, 1, -1], 1, 4))

In [12]:
@code_warntype frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

MethodInstance for frag_face2(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Int64}, ::Int64)
  from frag_face2(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, sp_idx::Vector{Int64}, sigma::Int64) in Main at c:\Users\Valerio\Desktop\TGW-3D.jl\notebooks\frag_face.ipynb:1
Arguments
  #self#::Core.Const(frag_face2)
  V::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
  sp_idx::Vector{Int64}
  sigma::Int64
Locals
  @_7::Any
  nvsize::Int64
  nFE::SparseMatrixCSC{Int8, Int64}
  nEV::SparseMatrixCSC{Int8, Int64}
  nV::Matrix{Float64}
  tV::Matrix{Float64}
  M::Matrix{Float64}
  sEV@_14::Core.Box
  sV@_15::Core.Box
  sigmavs::Vector{Int64}
  vs_num::Int64
  #9::var"#9#10"{SparseMatrixCSC{Int8, Int64}, SparseMatrixCSC{Int8, Int64}, Vector{Int64}, Int64, Matrix{Float64}}
  task::Task
  sV@_20::Union{}
  sV@_21::Union{}
  sV@_22::Union{}
  sEV@_23::Union{}
Body::Tuple{Matrix

getfield(sEV@_14, :contents)::Any
│    %114 = Core.apply_type(Main.SparseMatrixCSC, Main.Int8, Main.Int64)::Core.Const(SparseMatrixCSC{Int8, Int64})
│    %115 = Core.typeassert(%113, %114)::SparseMatrixCSC{Int8, Int64}
│    %116 = Main.length(sigmavs)::Int64
│    %117 = Main.ones(Main.Int8, %116)::Vector{Int8}
│    %118 = Main.sparsevec(%117)::SparseVector{Int8, Int64}
│    %119 = (%99)(%107, %115, %118)::Any
│    %120 = Base.indexed_iterate(%119, 1)::Any
│    %121 = Core.getfield(%120, 1)::Any
│           (@_7 = Core.getfield(%120, 2))
│    %123 = Base.indexed_iterate(%119, 2, @_7)::Any
│    %124 = Core.getfield(%123, 1)::Any
│           (@_7 = Core.getfield(%123, 2))
│    %126 = Base.indexed_iterate(%119, 3, @_7)::Any
│    %127 = Core.getfield(%126, 1)::Any
│    %128 = Core.apply_type(Main.Matrix, Main.Float64)::Core.Const(Matrix{Float64})
│    %129 = Base.convert(%128, %121)::Any
│           (nV = Core.typeassert(%129, %128))
│    %131 = Core.apply_type(Main.SparseMatrixCSC, Main.In

In [13]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [14]:
ProfileView.view()
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
  ╎49 @Base\client.jl:495; _start()
  ╎ 49 @Base\client.jl:292; exec_options(opts::Base.JLOptions)
  ╎  49 @Base\Base.jl:418; include(mod::Module, _path::String)
  ╎   49 @Base\loading.jl:1253; _include(mapexpr::Function, mod...
  ╎    49 @Base\loading.jl:1196; include_string(mapexpr::typeof...
  ╎     49 @Base\boot.jl:373; eval
  ╎    ╎ 49 ...otebook\notebook.jl:32; top-level scope
  ╎    ╎  49 ...c\serve_notebook.jl:75; (::VSCodeServer.var"#serve_not...
  ╎    ╎   49 ...\serve_notebook.jl:136; serve_notebook(pipename::Str...
  ╎    ╎    49 @JSONRPC\src\typed.jl:67; dispatch_msg(x::VSCodeServer...
  ╎    ╎     49 ...serve_notebook.jl:13; notebook_runcell_request(con...
  ╎    ╎    ╎ 49 ...erver\src\repl.jl:184; withpath(f::VSCodeServer.va...
  ╎    ╎    ╎  49 ...serve_notebook.jl:19; (::VSCodeServer.var"#164#16...
  ╎    ╎    ╎   49 @Base\essentials.jl:714; invokelatest
  ╎    ╎    ╎    49 ...e\essentials.jl:716; #invokelatest#


  ╎    ╎    ╎    ╎    ╎    ╎ 1  ...ctarray.jl:2849; map
  ╎    ╎    ╎    ╎    ╎    ╎  1  ...e\array.jl:653; collect_similar
  ╎    ╎    ╎    ╎    ╎    ╎   1  ...e\array.jl:754; _collect(c::Vect...
  ╎    ╎    ╎    ╎    ╎    ╎    1  ...\array.jl:760; collect_to_with_...
  ╎    ╎    ╎    ╎    ╎    ╎     1  ...\array.jl:782; collect_to!(des...
 1╎    ╎    ╎    ╎    ╎    ╎    ╎ 1  ...erator.jl:47; iterate
  ╎    ╎    ╎    ╎    ╎     1  ...l_cycles.jl:184; (::LinearAlgebraic...
  ╎    ╎    ╎    ╎    ╎    ╎ 1  ...evector.jl:1789; *(A::SparseMatrix...
  ╎    ╎    ╎    ╎    ╎    ╎  1  ...evector.jl:1685; densemv
 1╎    ╎    ╎    ╎    ╎    ╎   1  ...sevector.jl:0; densemv(A::SparseM...
  ╎    ╎    ╎    ╎    ╎     1  ...l_cycles.jl:186; (::LinearAlgebraic...
  ╎    ╎    ╎    ╎    ╎    ╎ 1  ...ctarray.jl:2849; map(f::Function, ...
 1╎    ╎    ╎    ╎    ╎     1  ...l_cycles.jl:188; (::LinearAlgebraic...
  ╎    ╎    ╎    ╎    ╎    1  ...al_cycles.jl:20; minimal_2cycles(V::...
  ╎    ╎    ╎    ╎   

In [15]:
@benchmark frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

BenchmarkTools.Trial: 8171 samples with 1 evaluation.
 Range (min … max):  376.300 μs … 59.363 ms  ┊ GC (min … max):  0.00% … 97.57%
 Time  (median):     436.200 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   607.541 μs ±  1.482 ms  ┊ GC (mean ± σ):  16.21% ±  7.21%

  ▁█▁                                                           
  ███▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  376 μs          Histogram: frequency by time         1.17 ms <

 Memory estimate: 265.71 KiB, allocs estimate: 4874.

In [16]:
@benchmark frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

BenchmarkTools.Trial: 9179 samples with 1 evaluation.
 Range (min … max):  297.300 μs … 35.314 ms  ┊ GC (min … max):  0.00% … 97.50%
 Time  (median):     376.200 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   540.377 μs ±  1.575 ms  ┊ GC (mean ± σ):  19.48% ±  6.66%

   ▆█▄                                                          
  ▄███▇▅▄▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  297 μs          Histogram: frequency by time          924 μs <

 Memory estimate: 218.65 KiB, allocs estimate: 3873.